In [ ]:
import pandas as pd
import os
import re
import csv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.decomposition import PCA
import plotly.express as px
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
import jieba

In [ ]:
#n-gram tokenizer without spaces
def n_gram_tokenizer(text, n = 1):
    return [text[i:i+n] for i in range(len(text) - (n - 1))]

#did we want this to be like this? ['我 对', '对 中', '中 文', '文 感', '感 兴', '兴 趣'], or all in one string

In [ ]:
texts = []

with open('texts.txt', 'r') as f:
    for i, line in enumerate(f, 1):
        text = line.strip()  # Remove leading/trailing whitespace and newline characters
        if not text:
            print(f"Warning: Empty text found on line {i}.")
        texts.append(text)

In [ ]:
genres = []

with open('genres.txt', 'r') as f:
    for i, line in enumerate(f, 1):
        genre = line.strip()  # Remove leading/trailing whitespace and newline characters
        if not text:
            print(f"Warning: Empty text found on line {i}.")
        genres.append(genre)

In [ ]:
def two_gram_tokenizer(text):
    return n_gram_tokenizer(text, n=2)

In [ ]:
chunk_size = 1000

vectorizer_2gram = TfidfVectorizer(max_features=100, analyzer='word', tokenizer=two_gram_tokenizer)

# Tokenize and transform texts in chunks
# PV: here is my proposed ammended code
all_chunk_texts = []
all_chunk_genres = []
for i, text in enumerate(texts): # for text in texts:
    chunk_texts = [text[j:j + chunk_size] for j in range(0, len(text), chunk_size)]
    chunk_genres = [genres[i] for _ in range(len(chunk_texts))]  # Assign the corresponding genre to each chunk
    all_chunk_texts.extend(chunk_texts)
    all_chunk_genres.extend(chunk_genres)

In [ ]:
def jieba_tokenizer(text):
    return ''.join(jieba.lcut(text))

In [ ]:
chunk_size = 1000

jieba_chunk_texts = []
jieba_chunk_genres = []
for i, text in enumerate(texts):
    chunk_tokens = jieba.lcut(text)  # Tokenize text using Jieba
    chunk_texts = [''.join(chunk_tokens[j:j + chunk_size]) for j in range(0, len(chunk_tokens), chunk_size)]
    chunk_genres = [genres[i] for _ in range(len(chunk_texts))]  # Assign the corresponding genre to each chunk
    jieba_chunk_texts.extend(chunk_texts)
    jieba_chunk_genres.extend(chunk_genres)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/2r/fj2_zqjj6mj4cldvkdp5_bx00000gs/T/jieba.cache
Loading model cost 0.568 seconds.
Prefix dict has been built successfully.


1 gram

In [ ]:
vectorizer = TfidfVectorizer(max_features=100, tokenizer=n_gram_tokenizer)
vectors = vectorizer.fit_transform(texts)

/Users/lola/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
X = vectors
y = genres

In [ ]:
clf = SGDClassifier(loss = 'hinge', max_iter = 5000)

In [ ]:
scores = cross_val_score(clf, X, y, cv=10)
print(scores)

/Users/lola/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


[0.79777236 0.80403759 0.81691612 0.81726418 0.80090498 0.80682214
 0.80536212 0.79735376 0.80571031 0.80118384]


2 gram

In [ ]:
import pickle

# Load data from the .p file
with open('2gramvectorizer.p', 'rb') as file:
    twogramvectorizer = pickle.load(file)

with open('2gramchunks.p', 'rb') as file:
    twogramchunks = pickle.load(file)

/var/folders/2r/fj2_zqjj6mj4cldvkdp5_bx00000gs/T/ipykernel_52238/1010760311.py:8: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  twogramchunks = pickle.load(file)


In [ ]:
X = twogramchunks
y = np.array(all_chunk_genres)

In [ ]:
scores = cross_val_score(clf, X, y, cv=10)
print(scores)

[0.65466096 0.70412623 0.72173324 0.68237112 0.65623487 0.6785302
 0.67306272 0.7002818  0.68376464 0.71413513]


Jieba

In [ ]:
import pickle

In [ ]:
import jieba

# Load data from the .p file
with open('jbvectorizer.p', 'rb') as file:
    jbvectorizer = pickle.load(file)

with open('jbchunks.p', 'rb') as file:
    jbchunks = pickle.load(file)

/var/folders/2r/fj2_zqjj6mj4cldvkdp5_bx00000gs/T/ipykernel_52238/4150266919.py:8: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  jbvectorizer = pickle.load(file)
/Users/lola/anaconda3/lib/python3.11/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/lola/anaconda3/lib/python3.11/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html

In [ ]:
X = jbchunks
y = np.array(all_chunk_genres)

In [ ]:
scores = cross_val_score(clf, X, y, cv=10)
print(scores)

[0.71315022 0.73677682 0.77117341 0.71547703 0.69327736 0.70838084
 0.69900904 0.7408236  0.71775622 0.74930267]
